In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

option = webdriver.ChromeOptions()
option.add_argument("headless")
ISBN = input()
# driver = webdriver.Chrome(path, options=option)
## path請改成各自電腦中chromedriver.exe的位置
driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver", options=option)


## 臺北市立圖書館
result = []
present = []
driver.get(f"https://book.tpml.edu.tw/webpac/bookSearchList.do?searchtype=simplesearch&execodeHidden=true&execode=&authoriz=1&search_field=ISBN&search_input={ISBN}")

re = requests.get(driver.current_url)
soup_result = BeautifulSoup(re.text, "html.parser")

if soup_result.find("div", class_ = "path").findAll("a")[2].text == "書目資料":
    trlist = driver.find_element_by_class_name("order").find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        tmp = [col.text for col in tdlist]
        tmp.insert(0, "臺北市立圖書館")
        result.append(tmp)
    result.remove(result[0])
    for i in range(len(result)):
        present_data = [result[i][0].strip(), result[i][1].strip(), result[i][5].strip(), driver.current_url]
        present.append(present_data)


else:
    driver.get(driver.current_url)
    driver.switch_to.frame("leftFrame")
    frametables = driver.find_element_by_class_name("tablesorter")
    hrefs = frametables.find_element_by_tag_name("a")
    hrefs.click()    
    trlist = driver.find_element_by_class_name("order").find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        tmp = [col.text for col in tdlist]
        tmp.insert(0, "臺北市立圖書館")
        result.append(tmp)
    result.remove(result[0])
    for i in range(len(result)):
        present_data = [result[i][0].strip(), result[i][1].strip(), result[i][5].strip(), driver.current_url]
        present.append(present_data)

## 新北市立圖書館
driver.get("https://webpac.tphcc.gov.tw/webpac/search.cfm")

keyword = driver.find_element_by_class_name("ui-autocomplete-input")
keyword.send_keys(ISBN)
time.sleep(0.5)

button = driver.find_element_by_class_name("search-btn")
button.click()
time.sleep(0.5)

link = driver.find_element_by_id("autolink")
href = link.get_attribute("href")
index = href.find("mid=") # 得到'm'的index整數值
id = href[index+4:index+10] # 得到六位數id

new_link = "https://webpac.tphcc.gov.tw/webpac/ajax_page/get_content_area.cfm?mid=" + id + "&i_list_number=250&i_page=1&i_sory_by=1"
re = requests.get(new_link) 
soup = BeautifulSoup(re.text, "html.parser")
table = soup.find("table")

for row in table.find_all("tr")[1:]:
    place = row.find("td", {"data-title":"典藏地名稱："}).span.text
    status = row.find("td", {"data-title":"館藏狀態："}).span.text
    place = place.strip()
    status = status.replace("\r\n", "")
    status = status.strip().split(" ")
    present_data = ["新北市立圖書館", place, status[0], href]
    present.append(present_data)
    
driver.get("https://metacat.ntu.edu.tw/") # 更改網址以前往不同網頁

element = driver.find_element_by_name('simpleSearchText')
element.send_keys(ISBN)
select = Select(driver.find_element_by_id('simpleType'))
select.select_by_value("ISBN")

# 把不要的勾掉
choose_btn = driver.find_element_by_link_text("機構單位篩選").click()
time.sleep(1)

btn_mid = driver.find_element_by_id('library1').click()
btn_south = driver.find_element_by_id('library2').click()
btn_east = driver.find_element_by_id('library3').click()

no_hsc = driver.find_element_by_id('hsc').click() # 新生醫專
no_tust = driver.find_element_by_id('tust').click() # 大華科大
no_must = driver.find_element_by_id('must').click() # 明新科大
no_taitheo = driver.find_element_by_id('taitheo').click() # 台灣神學研究學院
no_dila = driver.find_element_by_id('dila').click() # 法鼓文理學院
no_yzu = driver.find_element_by_id('yzu').click() # 元智大學
no_niu = driver.find_element_by_id('niu').click() # 宜蘭大學
no_lhu = driver.find_element_by_id('lhu').click() # 龍華科大
no_oit = driver.find_element_by_id('oit').click() # 亞東技術學院
no_ntuvvAlma = driver.find_element_by_id('ntuvvAlma').click() # 原住民圖資中心

save_opt = driver.find_element_by_id("saveOptions").click() # 儲存已選選項
driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕
close = driver.find_element_by_class_name("close").click() # 按叉叉

search_gogogo = driver.find_element_by_id('simpleSearchButton').click()
time.sleep(70)

# 有"顯示更多"就按下去
more = driver.find_elements_by_name('collapseLink')
for i in range(len(more)):
    more[i].click()
    time.sleep(1)

# 每頁顯示 100 項搜尋結果
try:
    show = Select(driver.find_element_by_name("resultTable_length"))
    show.select_by_value("100")
except:
    show = None
    

#爬 Metacat
if show != None:    
    name = []
    books = driver.find_elements_by_class_name('institution-list')
    for i in range(len(books)):
        name.append(books[i].text)

    URL = []
    www = driver.find_elements_by_class_name('institution-list')
    for i in range(len(www)):
        website = www[i].get_attribute('href')
        if website != None:
            URL.append(website)
        else:
            www[i].click()
            www2 = driver.find_elements_by_class_name('institution-list')
            URL.append(www2[-1].get_attribute('href'))
            ActionChains(driver).move_by_offset(150, 200).click().perform()
            ActionChains(driver).move_by_offset(-150, -200).perform()

    for w in URL:
        web = str(w)
        driver.get(web)
        time.sleep(8)
        if 'ntu.primo' in web: #台大系統
            try:
                time.sleep(5)
                try:
                    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
                except:
                    back = None
                if back != None:
                    back.click()
                thelist = driver.find_elements_by_class_namethelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
                for row in thelist:
                    plist = row.find_elements_by_tag_name("p")
                    where = row.find_elements_by_tag_name("h3")
                    i = len(where)
                    for sth in plist:
                        a = sth.find_elements_by_tag_name("span")
                        for _ in range(i): 
                            present_data = ['台灣大學', where[_].text.strip(), a[0].text.strip(), w]
                            present.append(present_data)                            
                            break
                        break
            except:
                present_data = ['台灣大學', 'Error', 'Error', w]
                present.append(present_data)
                

        elif 'ntnu' in web: #師大系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['臺灣師範大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)                    
            except:
                present_data = ['臺灣師範大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'nccu' in web: #政大系統
            try:
                time.sleep(5)
                try:
                    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
                except:
                    back = None
                if back != None:
                    back.click()
                thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
                for row in thelist:
                    plist = row.find_elements_by_tag_name("p")
                    where = row.find_elements_by_tag_name("h3")
                    i = len(where)
                    for sth in plist:
                        a = sth.find_elements_by_tag_name("span")
                        for _ in range(i): 
                            present_data = ['政治大學', where[_].text.strip(), a[0].text.strip(), w]
                            present.append(present_data)                    
                            break
                        break
            except:
                present_data = ['政治大學', 'Error', 'Error', w]
                present.append(present_data)                

        elif 'ntust' in web: #台科大系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['臺灣科技大學', tdlist[0].text.strip(), tdlist[5].text.strip(), w]
                    present.append(present_data)
                    
            except:
                present_data = ['臺灣科技大學', 'Error', 'Error', w]
                present.append(present_data)              

        elif 'ym' in web: #陽明系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['陽明大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)
                    
            except:
                present_data = ['陽明大學', 'Error', 'Error', w]
                present.append(present_data)                

        elif 'ntou' in web: #海大系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['海洋科技大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)                    
            except:
                present_data = ['海洋科技大學', 'Error', 'Error', w]
                present.append(present_data)              

        elif 'ntcu' in web: #交大系統
            try:
                time.sleep(5)
                try:
                    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
                except:
                    back = None
                if back != None:
                    back.click()
                thelist = driver.find_elements_by_class_name("tab-content-header.margin-bottom-small.margin-left-medium.layout-align-space-between-end.layout-row")
                for row in thelist:
                    plist = row.find_elements_by_tag_name("p")
                    where = row.find_elements_by_tag_name("h4")
                    i = len(where)
                    for sth in plist:
                        a = sth.find_elements_by_tag_name("span")
                        for _ in range(i):
                            present_data = ['交通大學', where[_].text.strip(), a[0].text.strip(), w]
                            present.append(present_data)
                            break
                        break
            except:
                present_data = ['交通大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'nthu' in web: #清華系統
            try:
                time.sleep(5)
                where = driver.find_elements_by_link_text('全部館藏')[1].click()
                time.sleep(5)
                table = driver.find_element_by_xpath("/html/body/table[12]")
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['清華大學', tdlist[2].text.strip(), tdlist[8].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['清華大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'ncu' in web: #中央系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['中央大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)
            except:
                present_data = ['中央大學', 'Error', 'Error', w]
                present.append(present_data)                

        elif 'ncl' in web: #國圖系統
            try:
                where = driver.find_element_by_link_text("書在哪裡(請點選)").click()
                time.sleep(2)
                tdlist = driver.find_elements_by_class_name('td1')
                present_data = ['國家圖書館', tdlist[1].text.strip(), tdlist[6].text.strip(), w]
                present.append(present_data)
            except:
                present_data = ['國家圖書館', 'Error', 'Error', w]
                present.append(present_data)
                
        elif 'las.sinica' in web: #中研院系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['中央研究院',tdlist[0].text.strip(), tdlist[2].text.strip(), w]
                    present.append(present_data)
            except:
                present_data = ['中央研究院', 'Error', 'Error', w]
                present.append(present_data)

        elif 'ntpu' in web: #北大系統
            try:
                table = driver.find_element_by_class_name("book_location")
                trlist = table.find_elements_by_class_name('odd')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['臺北大學', tdlist[2].text, w]
                        present.append(present_data)
                        break
            except:
                present_data = ['臺北大學', 'Error', 'Error', w]
                present.append(present_data)

        elif '203.64.5.158' in web: #台北藝術大學系統
            try:
                where = driver.find_element_by_xpath("/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a").click()
                time.sleep(2)
                table = driver.find_element_by_class_name('table.table-bordered')
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['臺北藝術大學', tdlist[0].text.strip(), tdlist[5].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['臺北藝術大學', 'Error', 'Error', w]
                present.append(present_data)

        elif '140.131.94.8' in web: #護理健康大學系統
            try:
                table = driver.find_element_by_xpath("/html/body/div[4]/div[1]/form[1]/div[2]/div[1]/ul[3]/li[1]/ul[3]/li[2]/table")
                trlist = table.find_elements_by_tag_name('tr')
                for _ in trlist:
                    tdlist = driver.find_elements_by_class_name('holdingslist')
                    present_data = ['臺北護理健康大學', '圖書館', tdlist[3].text.strip(), w]
                    present.append(present_data)
                    break
            except:
                present_data = ['臺北護理健康大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'ntut' in web: #北科系統
            try:
                table = driver.find_element_by_class_name('order')
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['臺北科技大學', tdlist[2].text.strip(), tdlist[8].text.strip(), w]
                        present.append(present_data)
                        break    
            except:
                present_data = ['臺北科技大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'ntl' in web: #國台圖系統
            try:
                where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
                time.sleep(2)
                tdlist = driver.find_elements_by_class_name('td1')
                present_data = ['國立臺灣圖書館', tdlist[8].text.strip(), tdlist[14].text.strip(), w]
                present.append(present_data)
            except:
                present_data = ['國立臺灣圖書館', 'Error', 'Error', w]
                present.append(present_data)

        elif 'itrilib' in web: #工研院系統
            try:
                where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/a").click()
                time.sleep(2)
                tdlist = driver.find_elements_by_class_name('td1')
                present_data = ['工業研究院', tdlist[10].text.strip(), tdlist[12].text.strip(), w]
                present.append(present_data)
            except:
                present_data = ['工業研究院', 'Error', 'Error', w]
                present.append(present_data)

        elif 'nhri' in web: #國衛院系統
            try:
                where = driver.find_element_by_xpath('/html/body/table[8]/tbody/tr[1]/td[2]/a').click()
                time.sleep(2)
                tdlist = driver.find_elements_by_class_name('td1')
                present_data = ['國家衛生研究院', tdlist[7].text.strip(), tdlist[13].text.strip(), w]
                present.append(present_data)
            except:
                present_data = ['國家衛生研究院', 'Error', 'Error', w]
                present.append(present_data)

        elif 'webpac.pccu' in web: #中國文化大學系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['中國文化大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)
            except:
                present_data = ['中國文化大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'shu' in web: #世新系統
            try:
                where = driver.find_element_by_link_text("世新圖書館").click()
                table = driver.find_element_by_xpath("/html/body/p[2]/table[1]")
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['世新大學', tdlist[1].text.strip(), tdlist[3].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['世新大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'uco-network' in web: #淡江系統
            try:
                time.sleep(5)
                try:
                    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
                except:
                    back = None
                if back != None:
                    back.click()
                thelist = driver.find_elements_by_class_name("neutralized-button.layout-full-width.layout-display-flex.md-button.md-ink-ripple.layout-row")
                for row in thelist:
                    plist = row.find_elements_by_tag_name("p")
                    where = row.find_elements_by_tag_name("h3")
                    i = int(len(where))
                    for sth in plist:
                        a = sth.find_elements_by_tag_name("span")
                        for _ in range(i): 
                            present_data = ['淡江大學', where[_].text.strip(), a[0].text.strip(), w]
                            present.append(present_data)
                            break
                        break
            except:
                present_data = ['淡江大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'fju' in web: #輔仁大學系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['輔仁大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)
            except:
                present_data = ['輔仁大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'mcu' in web: #銘傳系統
            try:
                time.sleep(5)
                try:
                    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
                except:
                    back = None
                if back != None:
                    back.click()
                thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
                for row in thelist[:-2]:
                    plist = row.find_elements_by_tag_name("p")
                    where = row.find_elements_by_tag_name("h3")
                    i = len(where)
                    for sth in plist:
                        a = sth.find_elements_by_tag_name("span")
                        for _ in range(1): 
                            present_data = ['銘傳大學', where[_].text.strip(), a[0].text.strip(), w]
                            present.append(present_data)
                            break
                        break
            except:
                present_data = ['銘傳大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'scu' in web: #東吳系統
            try:
                time.sleep(5)
                try:
                    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
                except:
                    back = None
                if back != None:
                    back.click()
                thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
                for row in thelist[:-2]:
                    plist = row.find_elements_by_tag_name("p")
                    where = row.find_elements_by_tag_name("h3")
                    i = len(where)
                    for sth in plist:
                        a = sth.find_elements_by_tag_name("span")
                        for _ in range(1): 
                            present_data = ['東吳大學', where[_].text.strip(), a[0].text.strip(), w]
                            present.append(present_data)
                            break
                        break 
            except:
                present_data = ['東吳大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'cylis' in web: #中原大學系統
            try:
                trlist = driver.find_elements_by_class_name('bibItemsEntry')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    present_data = ['中原大學', tdlist[0].text.strip(), tdlist[3].text.strip(), w]
                    present.append(present_data)
            except:
                present_data = ['中原大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'tmu' in web: #北醫系統
            try:
                table = driver.find_element_by_class_name('order')
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['臺北醫學大學', tdlist[2].text.strip(), tdlist[5].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['臺北醫學大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'moc' in web: #台博館系統
            try:
                where = driver.find_element_by_link_text("書在哪裡 ? (請點我)").click()
                table = driver.find_element_by_xpath("/html/body/table[9]")
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['國立臺灣博物館', tdlist[3].text.strip(), tdlist[7].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['國立臺灣博物館', 'Error', 'Error', w]
                present.append(present_data)

        elif 'jinwen' in web: #景文科大系統
            try:
                table = driver.find_element_by_class_name('order')
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['景文科技大學', tdlist[2].text.strip(), tdlist[5].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['景文科技大學', 'Error', 'Error', w]
                present.append(present_data)

        elif 'mmc' in web: #馬偕醫護系統
            try:
                where = driver.find_element_by_xpath("/html/body/table[8]/tbody/tr[1]/td[2]/span/a[1]").click()
                time.sleep(2)
                tdlist = driver.find_elements_by_class_name('td1')
                present_data = ['馬偕醫護管理專科學校', tdlist[7].text.strip(), tdlist[12].text.strip(), w]
                present.append(present_data)
            except:
                present_data = ['馬偕醫護管理專科學校', 'Error', 'Error', w]
                present.append(present_data)

        elif 'tfi' in web: #視聽文化中心系統
            try:
                table = driver.find_element_by_xpath("/html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div[3]/div/div[4]")
                trlist = table.find_elements_by_tag_name('tr')
                for row in trlist:
                    tdlist = row.find_elements_by_tag_name('td')
                    for sth in tdlist:
                        present_data = ['國家視聽文化中心', tdlist[1].text.strip(), tdlist[4].text.strip(), w]
                        present.append(present_data)
                        break
            except:
                present_data = ['國家視聽文化中心', 'Error', 'Error', w]
                present.append(present_data)
    
wb = Workbook()
dest_filename = 'UserInfoFile.xlsx'

ws1 = wb.active
titleList = ['圖書館', '分館資訊', '館藏位置', '網址']

for row in range(len(titleList)):
    c = row + 1 
    ws1.cell(row=1, column=c, value=titleList[row])
    
for listIndex in range(len(present)):
    ws1.append(present[listIndex]) 
wb.save(filename=dest_filename)
       
    

9789869109321


<ipython-input-2-ba2a56fa2f16>:113: DeprecationWarning: use driver.switch_to.alert instead
  driver.switch_to_alert().accept() # 點選彈出裡面的確定按鈕
